In [1]:
import os
import shutil
import subprocess 
import itertools
import numpy as np

import mlflow
from random import random, randint
from mlflow import log_metric, log_param, log_artifact, log_artifacts

mlflow.set_experiment("selection-8-o2")

2022/03/30 13:17:39 INFO mlflow.tracking.fluent: Experiment with name 'selection-8-o2' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///files0/extended/bohm747/Development/soda/soda-opt/experiments/selection/mlruns/5', experiment_id='5', lifecycle_stage='active', name='selection-8-o2', tags={}>

In [2]:
# The soda opt pipeline (simplified)
sodaopt_optimization_options = [
    ('-convert-linalg-to-affine-loops', 'mandatory'),
    # ('-affine-data-copy-generate="generate-dma=false fast-mem-space=0"', 'optional'),
    # ('-erase-buffer-deallocation', 'optional'),
    # ('-promote-buffers-to-stack="max-rank-of-allocated-memref=4 max-alloc-size-in-bytes=4096"', 'optional'),
    ('-affine-data-copy-generate="generate-dma=false fast-mem-space=0" -erase-buffer-deallocation -promote-buffers-to-stack="max-rank-of-allocated-memref=4 max-alloc-size-in-bytes=4096"', 'optional'),
    # ('-affine-loop-unroll="unroll-full" -affine-loop-unroll="unroll-full" -affine-loop-unroll="unroll-full"', 'optional'),
    ('-affine-loop-unroll="unroll-full"', 'optional'),
    ('-affine-loop-unroll="unroll-full"', 'optional'),
    ('-affine-loop-unroll="unroll-full"', 'optional'),
    ('-affine-scalrep', 'optional'),
    ('-cse', 'optional'),
    ('-lower-affine -convert-scf-to-cf -convert-memref-to-llvm', 'mandatory'),
    ('--convert-math-to-llvm --convert-math-to-libm', 'mandatory'),
    ('-arith-expand', 'mandatory'),
    ('-memref-expand', 'mandatory'),
    ('--convert-arith-to-llvm', 'mandatory'),
    ('-convert-std-to-llvm=use-bare-ptr-memref-call-conv', 'mandatory'),
    ('-reconcile-unrealized-casts', 'mandatory')]

# sodaopt_optimization_options = [
#     ('-convert-linalg-to-affine-loops', 'mandatory'),
#     ('-affine-data-copy-generate="generate-dma=false fast-mem-space=0"', 'mandatory'),
#     ('-erase-buffer-deallocation', 'mandatory'),
#     ('-promote-buffers-to-stack="max-rank-of-allocated-memref=4 max-alloc-size-in-bytes=4096"', 'mandatory'),
#     ('-affine-loop-unroll="unroll-full" -affine-loop-unroll="unroll-full" -affine-loop-unroll="unroll-full"', 'mandatory'),
#     ('-affine-scalrep', 'mandatory'),
#     ('-cse', 'mandatory'),
#     ('-lower-affine -convert-scf-to-cf -convert-memref-to-llvm', 'mandatory'),
#     ('--convert-math-to-llvm --convert-math-to-libm', 'mandatory'),
#     ('-arith-expand', 'mandatory'),
#     ('-memref-expand', 'mandatory'),
#     ('--convert-arith-to-llvm', 'mandatory'),
#     ('-convert-std-to-llvm=use-bare-ptr-memref-call-conv', 'mandatory'),
#     ('-reconcile-unrealized-casts', 'mandatory')]


In [3]:
def getFullString(options, combinations):
  """Takes a list of options and the desired combination and output a string with selected options."""
  options_list = [option[0] for option, use_now in zip(options, combinations) if use_now == True]
  return ' '.join(options_list)


def generateCombinations(options):
  '''options is a list of tuples where the second word states mandatory or optional'''
  mandatory_truth_list = [v[1]=='mandatory' for v in options]

  indices_of_optional= [i for i, x, in enumerate(mandatory_truth_list) if x == False]

  possibilities = list(itertools.product([True,False], repeat=len(indices_of_optional)))

  
  combinations = []

  for p in possibilities:
    extended_possibility_list=np.array(mandatory_truth_list)
    extended_possibility_list[indices_of_optional]=p
    combinations.append(getFullString(options, extended_possibility_list))

  return combinations


optimization_combinations = generateCombinations(sodaopt_optimization_options)

optimization_combinations.insert(0,'-lower-all-to-llvm="use-bare-ptr-memref-call-conv"')
optimization_combinations.insert(0,'-soda-opt-pipeline-for-bambu="use-bare-ptr-memref-call-conv"')

print(len(optimization_combinations))

# optimization_combinations = []
# optimization_combinations.append('-lower-all-to-llvm="use-bare-ptr-memref-call-conv"')



66


In [4]:

for optimization in optimization_combinations:

  with mlflow.start_run() as run:

    # Prepare output folders 
    cwd=os.getcwd()
    output_root=os.path.join(cwd,'output')
    bambu_root=os.path.join(output_root,'bambu')

    try:
      shutil.rmtree(output_root)
    except OSError as e:
      print("Error: %s : %s" % (output_root, e.strerror))

    output_paths= [output_root, bambu_root]

    for d in output_paths:
        if not os.path.exists(d):
            os.makedirs(d)

    # SODA-OPT
    subprocess.run('cp kernel.mlir output/00_source.mlir', shell=True)
    subprocess.run('soda-opt --convert-affine-for-to-soda output/00_source.mlir -o output/01_searched.mlir', shell=True)
    subprocess.run('soda-opt --soda-outline-bambu-code --soda-extract-arguments-to-xml=using-bare-ptr output/01_searched.mlir -o output/02_outlined.mlir', shell=True)
    subprocess.run('mv main_kernel_test.xml output/bambu/main_kernel_test.xml', shell=True)
    subprocess.run('mv main_kernel_interface.xml output/bambu/main_kernel_interface.xml', shell=True)
    subprocess.run('soda-opt -soda-generate-bambu-accelcode=no-aa output/02_outlined.mlir -o output/03_isolated.mlir', shell=True)
    sodaopt_string='soda-opt {} output/03_isolated.mlir -o output/05_optimized.mlir -print-ir-before-all 2>&1 | cat > output/04_intermediate.mlir'.format(optimization)
    subprocess.run(sodaopt_string, shell=True)
    subprocess.run('mlir-translate --mlir-to-llvmir output/05_optimized.mlir -o output/06_llvm.ll', shell=True)
    subprocess.run('opt-10 --strip-debug output/06_llvm.ll -S -o output/07_model.ll', shell=True)
    subprocess.run('cp output/07_model.ll output/bambu/input.ll', shell=True)

    # BAMBU
    os.chdir(bambu_root)
    bambu_string='bambu -v3 --print-dot -lm --soft-float --compiler=I386_CLANG10 -O2 --device=nangate45 --clock-period=2 --no-iob --experimental-setup=BAMBU-BALANCED-MP --channels-number=2 --memory-allocation-policy=ALL_BRAM --disable-function-proxy --generate-tb=main_kernel_test.xml --simulate --simulator=VERILATOR --top-fname=main_kernel input.ll'
    bambu_cmd='docker run -u $(id -u):$(id -g) -v $(pwd):/working_dir --rm soda/soda {} 2>&1 | cat > bambu-exec-log'.format(bambu_string)
    subprocess.run(bambu_cmd, shell=True)
    os.chdir(cwd)


    # Retrieve info
    result = 0 
    completed=False
    metrics = {}

    with open(os.path.join(bambu_root, 'bambu-exec-log')) as f:
      for l in f.readlines():
        if ('SUCCESS' in l):
          completed=True

        if ('Total cycles' in l):
          result = int(l.split(sep=':')[1].split()[0])

    if(completed):
      mlflow.set_tag("LOG_STATUS", "SUCCESS")
    else:
      mlflow.set_tag("LOG_STATUS", "FAIL")


    metrics={'cycles': result}

    artifacts = [
      os.path.join(output_root, '00_source.mlir'),
      os.path.join(output_root, '01_searched.mlir'),
      os.path.join(output_root, '02_outlined.mlir'),
      os.path.join(output_root, '03_isolated.mlir'),
      os.path.join(output_root, '04_intermediate.mlir'),
      os.path.join(output_root, '05_optimized.mlir'),
      os.path.join(output_root, '06_llvm.ll'),
      os.path.join(output_root, '07_model.ll'),
      os.path.join(bambu_root,  'bambu-exec-log'),
      os.path.join(bambu_root,  'main_kernel.v'),
      os.path.join(bambu_root,  'HLS_output/dot/main_kernel/HLS_STGraph.dot')
    ]


    # Log info
    log_param("sodaopt_string", sodaopt_string)
    log_param("bambu_string", bambu_string)
    for artifact in artifacts:
      try:
        log_artifact(artifact)
      except FileNotFoundError as e:
        print("Warning: %s : %s" % (artifact, e.strerror))

    # bambu has a symlink, so this is not possible
    # log_artifacts(output_root)

    if (completed):
      for k,v in metrics.items():
        log_metric(k, v)
